<a href="https://colab.research.google.com/github/Vikaash07/machine-learning/blob/main/brain_tumor_E0122049.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CONNECTING DRIVE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import itertools
from PIL import Image

In [ ]:
# import handling data tools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
# import Deep Learning Libs
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
from PIL import Image

In [ ]:
# Generate Data paths with labels (TRAIN)

train_data_dir = '/content/drive/MyDrive/brain tumor data/Training'
filepaths = []
labels = []

folds = os.listdir(train_data_dir)
for fold in folds:
    foldpath = os.path.join(train_data_dir, fold)
    filelist = os.listdir(foldpath)
    for file in filelist:
        fpath = os.path.join(foldpath, file)

        filepaths.append(fpath)
        labels.append(fold)

In [ ]:
# Concatenate Data Paths with Labels into one Dataframe

FSeries = pd.Series(filepaths, name='filepaths')
LSeries = pd.Series(labels, name='labels')

train_df = pd.concat([FSeries, LSeries], axis=1)

In [ ]:
# Generate Data paths with labels (TEST)

test_data_dir = '/content/drive/MyDrive/brain tumor data/Testing'
filepaths = []
labels = []

folds = os.listdir(test_data_dir)
for fold in folds:
    foldpath = os.path.join(test_data_dir, fold)
    filelist = os.listdir(foldpath)
    for file in filelist:
        fpath = os.path.join(foldpath, file)

        filepaths.append(fpath)
        labels.append(fold)

In [ ]:
FSeries = pd.Series(filepaths, name='filepaths')
LSeries = pd.Series(labels, name='labels')

ts_df = pd.concat([FSeries, LSeries], axis=1)

In [ ]:
FSeries

In [ ]:
LSeries

In [ ]:
train_df

In [ ]:
# valid and test dataframe
valid_df, test_df = train_test_split(ts_df,  train_size= 0.5, shuffle= True, random_state= 123)

In [ ]:
test_df

In [ ]:
# crobed image size
batch_size = 16
img_size = (224, 224)

In [ ]:
tr_gen = ImageDataGenerator()
ts_gen = ImageDataGenerator()

In [ ]:
train_gen = tr_gen.flow_from_dataframe( train_df, x_col= 'filepaths', y_col= 'labels', target_size= img_size, class_mode= 'categorical',color_mode= 'rgb', shuffle= True, batch_size= batch_size)

In [ ]:
valid_gen = ts_gen.flow_from_dataframe( valid_df, x_col= 'filepaths', y_col= 'labels', target_size= img_size, class_mode= 'categorical',color_mode= 'rgb', shuffle= True, batch_size= batch_size)

In [ ]:
test_gen = ts_gen.flow_from_dataframe( test_df, x_col= 'filepaths', y_col= 'labels', target_size= img_size, class_mode= 'categorical',color_mode= 'rgb', shuffle= False, batch_size= batch_size)

In [ ]:
g_dict = train_gen.class_indices      # defines dictionary {'class': index}
classes = list(g_dict.keys())       # defines list of dictionary's kays (classes), classes names : string
images, labels = next(train_gen)      # get a batch size samples from the generator

In [ ]:
plt.figure(figsize= (20, 20))

for i in range(16):
    plt.subplot(4, 4, i + 1)
    image = images[i] / 255       # scales data to range (0 - 255)
    plt.imshow(image)
    index = np.argmax(labels[i])  # get image index
    class_name = classes[index]   # get class of image
    plt.title(class_name, color= 'blue', fontsize= 12)
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels * BasicBlock.expansion, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels * BasicBlock.expansion)
        self.shortcut = nn.Sequential()

        if stride != 1 or in_channels != out_channels * BasicBlock.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * BasicBlock.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * BasicBlock.expansion)
            )

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += self.shortcut(x)
        out = self.relu(out)
        return out

In [ ]:
class ResNet18(nn.Module):
    def __init__(self, block, num_blocks, num_classes=1000):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [ ]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
# Load datasets
train_dataset = ImageFolder(root='/content/drive/MyDrive/brain tumor data/Training/', transform=transform)
valid_dataset = ImageFolder(root='/content/drive/MyDrive/brain tumor data/Testing/', transform=transform)

In [ ]:
# Data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False, num_workers=4)

In [ ]:
# Initialize model
model = ResNet18(BasicBlock, [2, 2, 2, 2], num_classes=len(train_dataset.classes)).to(device)

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training function with plotting
def train_and_save_model(model, criterion, optimizer, train_loader, valid_loader, num_epochs=15, model_path='best_model.pth'):
    best_valid_acc = 0.0
    training_losses = []
    training_accuracies = []
    validation_losses = []
    validation_accuracies = []

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        correct_train = 0
        total_train = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()
        train_accuracy = correct_train / total_train
        training_losses.append(train_loss / len(train_loader))
        training_accuracies.append(train_accuracy)

        model.eval()
        valid_loss = 0.0
        correct_valid = 0
        total_valid = 0
        with torch.no_grad():
            for images, labels in valid_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                valid_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total_valid += labels.size(0)
                correct_valid += (predicted == labels).sum().item()
        valid_accuracy = correct_valid / total_valid
        validation_losses.append(valid_loss / len(valid_loader))
        validation_accuracies.append(valid_accuracy)

        if valid_accuracy > best_valid_acc:
            best_valid_acc = valid_accuracy
            torch.save(model.state_dict(), model_path)
        print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss / len(train_loader):.4f}, Training Acc: {train_accuracy:.4f}, Valid Loss: {valid_loss / len(valid_loader):.4f}, Valid Acc: {valid_accuracy:.4f}')

    # Plotting
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(training_losses, label='Training Loss')
    plt.plot(validation_losses, label='Validation Loss')
    plt.title('Loss')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(training_accuracies, label='Training Accuracy')
    plt.plot(validation_accuracies, label='Validation Accuracy')
    plt.title('Accuracy')
    plt.legend()
    plt.show()

In [ ]:
train_and_save_model(model, criterion, optimizer, train_loader, valid_loader, num_epochs=15, model_path='best_model.pth')

Epoch 1/15, Training Loss: 0.8345, Training Acc: 0.6705, Valid Loss: 0.6924, Valid Acc: 0.7246
Epoch 2/15, Training Loss: 0.5945, Training Acc: 0.7687, Valid Loss: 0.4983, Valid Acc: 0.7933
Epoch 3/15, Training Loss: 0.4860, Training Acc: 0.8153, Valid Loss: 0.9079, Valid Acc: 0.6796
Epoch 4/15, Training Loss: 0.3751, Training Acc: 0.8618, Valid Loss: 0.5303, Valid Acc: 0.7971
Epoch 5/15, Training Loss: 0.3164, Training Acc: 0.8800, Valid Loss: 0.2976, Valid Acc: 0.8810
Epoch 6/15, Training Loss: 0.2587, Training Acc: 0.9076, Valid Loss: 0.3575, Valid Acc: 0.8764
Epoch 7/15, Training Loss: 0.2123, Training Acc: 0.9199, Valid Loss: 0.2462, Valid Acc: 0.9214
Epoch 8/15, Training Loss: 0.1725, Training Acc: 0.9355, Valid Loss: 0.3346, Valid Acc: 0.8879
Epoch 9/15, Training Loss: 0.1413, Training Acc: 0.9516, Valid Loss: 0.6378, Valid Acc: 0.7963


In [ ]:
# Function to load a model
def load_model(model_path='best_model.pth'):
    model = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=len(train_dataset.classes))
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
    return model

In [ ]:
# Function for prediction
def predict_image(model, image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)  # Add batch dimension
    image = image.to(device)
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs.data, 1)
    return predicted.item()

In [ ]:
# Example prediction
model = load_model('best_model.pth')
image_path = '/content/drive/MyDrive/brain tumor data/Testing/glioma/Te-glTr_0000.jpg'  # Update this path
predicted_class = predict_image(model, image_path)
print(f'Predicted class: {train_dataset.classes[predicted_class]}')